In [2]:
# Test the script run_LR_par.py

In [3]:
import numpy as np
import os
import scanpy as sc
import pandas as pd
import scipy.spatial as spatial
from multiprocessing import Pool

In [2]:
def LR_calcs(pair, bead_min = 10):
    
    print(pair,'started')
    
    ligand_centered=True # Whether the permutation test is based around ligand
    N=1000 # No. of permutations
    results=[];discarded_LR=[]
    for puck in datasets.index: # For each puck in a list of puck names, call the given ligand-receptor pair, open the puck, 
        
        ligand=pair.split('--')[0].split('_')
        receptor=pair.split('--')[1].split('_')
        
        adata=sc.read(f'{mydir}/{puck}.h5ad')
        ## Chloes edit
        sc.pp.calculate_qc_metrics(adata, percent_top=None, inplace=True)
        
        # print(adata.shape)
        bead_thresh=np.quantile(adata.obs.n_genes_by_counts,.25) # Calculate the 25th quantile of the number of genes in each cell
        bead_thresh=np.max([bead_thresh,150]) # return either the 25th quantile or 150, whichever is higher and use it as a filter for the minimum no. of genes
        sc.pp.filter_cells(adata, min_genes=bead_thresh)
        
        # Remove beads annotated as "Empty" (only applicable for pucks with area annotations)
        if 'area' in adata.obs:
            adata = adata[adata.obs.area!="Empty"].copy()
        
        sc.pp.filter_genes(adata, min_cells=10) # return genes seen in at least 10 cells
        sc.pp.calculate_qc_metrics(adata, percent_top=None, inplace=True)

        #fig, ax = plt.subplots(nrow, ncol, figsize=(ncol*wd,nrow*wd), gridspec_kw={'wspace':0.2})
        #axr=ax.ravel()
        
        ## Chloes edit
        spatial_obsm = np.column_stack((adata.obs['x'], adata.obs['y']))
        adata.obsm['spatial'] = spatial_obsm
        
        puck_locs=adata.obsm['spatial'] 
        
        # Plot all beads used in this analysis
        # sc.pl.spatial(adata, color = "area", spot_size = 20)

        #ligand_counts=adata[(adata[:,ligand].X>0),ligand].X.todense()
        #receptor_counts=adata[(adata[:,receptor].X>0),receptor].X.todense()

        if len(ligand)>1: # The number of genes associated with the ligand.
            try:
                shares=list( set(adata.var.index) & set(ligand) ) # if the ligands are found in the pucks filtered genes
                ad=adata[:,shares].copy() # subset object for the ligand gene(s)
                sc.pp.filter_cells(ad, min_counts=1) # subset object for cells with the ligand?
                qun=np.quantile(ad.obs.n_counts,.75) # get 75th quantile of UMI counts of that gene per cell 
                sc.pp.filter_cells(ad, min_counts=qun) # subset object for cells with higher than the 75th quantile of umi counts for that gene
                ligand_locs=ad.obsm['spatial'] 
            except:
                ligand_locs=np.array([])

        else:
            try:
                gene_counts=adata[adata[:,ligand].X>0][:,ligand].X.todense().tolist() # obtain all non-zero UMI counts for the ligand across all cells?
                thresh=np.quantile(gene_counts,.75) # get 75th quantile of UMI counts for that ligand?
                ligand_locs=adata[ (adata[:,ligand].X>=thresh)].obsm['spatial'] # subset for cells w/ ligand expressing above the 75th quantile
            except:
                ligand_locs=np.array([])
                
        if len(receptor)>1: 
            try:
                shares=list( set(adata.var.index) & set(receptor) ) 
                ad=adata[:,shares].copy()
                #sc.pp.filter_cells(ad, min_genes=1)
                sc.pp.filter_cells(ad, min_counts=1)
                qun=np.quantile(ad.obs.n_counts,.75)
                sc.pp.filter_cells(ad, min_counts=qun)
                receptor_locs=ad.obsm['spatial']
            except:
                receptor_locs=np.array([])
            
        else:
            try:
                gene_counts=adata[adata[:,receptor].X>0][:,receptor].X.todense().tolist()
                thresh=np.quantile(gene_counts,.75)
                receptor_locs=adata[ (adata[:,receptor].X>=thresh)].obsm['spatial']
            except:
                receptor_locs=np.array([])
                
        #tumor_tcr_locs=ad_tcr[ad_tcr.obs.tumor_beads==1].obsm['spatial']
        #print(pair,puck,receptor_locs.shape[0],ligand_locs.shape[0])
        
        if receptor_locs.shape[0]>bead_min and ligand_locs.shape[0]>bead_min: # more than 20 (or input value) cells containing the receptor and the ligand
            
            # to speed up we can switch whether to loop on ligand or receptor based on how many cells express the ligand or receptor
            if ligand_locs.shape[0]<receptor_locs.shape[0]: 
                ligand_centered=True
            else:
                ligand_centered=False
                
            for j, d in enumerate([15,30,100,300]): 
                # Return type: <class 'enumerate'>
                # [(0, '15'), (1, '30'), (2, '100'), (3, '300)]
                
                if ligand_centered:
                    #print('ligand_centered')
                    point_tree = spatial.cKDTree(receptor_locs) # cKDTree: kd-tree for quick nearest-neighbor lookup
                    t_N=point_tree.query_ball_point(ligand_locs, d) # query_ball_point: Find all points within distance r of point(s) x
                    true=sum([len(n) for n in t_N]) # true holds the number of ligands next to a receptor (at current distance)
                    counts=[] # counts is a vector holding the number of permuted ligands next to a receptor (at current distance)
                    for i in range(N):
                        fake_ligands=puck_locs[np.random.choice(puck_locs.shape[0], size=ligand_locs.shape[0], replace=False), :]
                        t_N=point_tree.query_ball_point(fake_ligands, d)
                        counts.append(sum([len(n) for n in t_N])) 
                else:
                    
                    #print('receptor_centered')
                    point_tree = spatial.cKDTree(ligand_locs)
                    t_N=point_tree.query_ball_point(receptor_locs, d)
                    true=sum([len(n) for n in t_N])
                    counts=[]
                    for i in range(N):
                        fake_receptors=puck_locs[np.random.choice(puck_locs.shape[0], size=receptor_locs.shape[0], replace=False), :]
                        t_N=point_tree.query_ball_point(fake_receptors, d)
                        counts.append(sum([len(n) for n in t_N]))
                        

                counts=np.array(counts)
                #min_c=counts.min();max_c=counts.max()
                #y, x, _ = axr[f].hist(counts, range=(min_c,max_c), bins=int(max_c-min_c)+1)

                #print(sum(counts),true)
                pval= np.round( ( sum(counts>true)+1 ) / (N+1), 7)

                #plt.title('Sum of number of ERV beads at '+str(d)+' pixels radius of each TCR')
                #plt.title('Sum of all ERV exressing beads \n
                #          in 'str(d)+' pixels radius of each TCR vs random shuffles of N locations from')
                #axr[f].set_title(pair + '  ' + '\n'+str(d)+' pixels radius'+ '\n'+'p = '+str(pval) )
                #axr[f].axvline(true, 0, ymax=y.max(), color='r')

                pid=puck
                
                # Chloes edit
                patient = "placeholder"
                puck_age = 0
                #patient=datasets.loc[puck].patient
                #puck_age=int(pid[:4])

                results.append([pair,pid,patient,pval,true,np.median(counts),np.mean(counts),np.std(counts),d,receptor_locs.shape[0],ligand_locs.shape[0]])
        else:
            discarded_LR.append([pair,puck,receptor_locs.shape[0],ligand_locs.shape[0]])
    
    try:
        res=pd.DataFrame(results)
        res.columns=['LR','puck','patient','pval','hits','median','mean','std','distance','receptor_beads','ligand_beads']
        #print(res)
        res.to_csv(f'{mydir}/{output_folder}/results_{pair}.csv')
    except:
        #print(pair,'finished')
        print(f'pair {pair} has no sample')
    try:
        dis=pd.DataFrame(discarded_LR)
        dis.columns=['LR','puck','receptor_beads','ligand_beads']
        dis.to_csv(f'{mydir}/{output_folder}/discard_{pair}.csv')
    except:
        #print(pair,'finished')
        print(f'pair {pair} has all samples')
    print(f'{pair} IS DONE')

In [4]:
datasets=pd.read_csv('puck_list_040225.csv',index_col=0)
datasets

,total_UMI
Puck,
02_Puck10_102TA_TCR_RNA_area_111124,1
02_Puck17_102TA_TCR_RNA_area_111124,1
02_Puck33_102TA_TCR_RNA_area_111124,1
02_Puck39_102TA_TCR_RNA_area_111224,1
02_Puck15_107TA_TCR_RNA_area_111124,1
02_Puck18_107TA_TCR_RNA_area_111124,1
02_Puck20_107TA_TCR_RNA_area_111124,1
02_Puck26_108TA_TCR_RNA_area_111124,1
02_Puck29_108TA_TCR_RNA_area_111124,1


In [5]:
LRs=pd.read_csv('LRs_to_investigate.csv',index_col=0)
LRs['pair']=LRs['ligand.complex']+'--'+LRs['receptor.complex']
pairs=pd.DataFrame(LRs['pair'].unique())
pairs.columns=['LR']
pairs['ligand']=pairs['LR'].apply(lambda x: x.split('--')[0])
pairs['receptor']=pairs['LR'].apply(lambda x: x.split('--')[1])
pairs.set_index('LR',inplace=True)

In [6]:
datasets=datasets.sort_values(by='total_UMI',ascending=False)

In [24]:
# RUN FOR ALL AREAS (except empty) WITH 10 BEAD MIN
mydir = "04_liana/"
output_folder = "output_10_bead_min_040225"

pool = Pool(60)
results = pool.map(LR_calcs, pairs.index)
pool.close()

GRN--TNFRSF1BGZMA--F2RGZMA--PARD3  HBEGF--CD82  startedstartedstarted
HGF--ITGB1

HMGB1--CD163HSP90B1--TLR7startedHGF--CD44ICAM1--ITGAL_ITGB2IL16--CD4IFNG--IFNGR1_IFNGR2ICAM1--SPNLCK--CD8A_CD8B  HMGB1--HAVCR2
 HBEGF--CD44 ICAM1--IL2RGLGALS3--LAG3LGALS9--CD44LTA--TNFRSF1B   LGALS1--CD69LTA_LTB--LTBRstartedLGALS3BP--ITGB1 LGALS9--HAVCR2LGALS9--CD47  startedLAMA5--CD44 startedLRPAP1--LRP1IL7--IL2RG_IL7Rstartedstarted   
  
 LAMA5--ITGA6_ITGB1
 startedstartedstarted
startedstartedstartedstartedstarted

 
started

  
started started  startedstarted

started



startedstartedstartedstarted



started



LTA--TNFRSF1B IS DONE
IFNG--IFNGR1_IFNGR2 IS DONE
LGALS3--LAG3 IS DONE
HGF--ITGB1 IS DONE
HBEGF--CD82 IS DONE
HGF--CD44 IS DONE
LGALS1--CD69 IS DONE
HBEGF--CD44 IS DONE
ICAM1--IL2RG IS DONE
IL7--IL2RG_IL7R IS DONE
LGALS9--CD44 IS DONE
LGALS9--HAVCR2 IS DONE
ICAM1--SPN IS DONE
LCK--CD8A_CD8B IS DONE
LGALS9--CD47 IS DONE
IL16--CD4 IS DONE
HSP90B1--TLR7 IS DONE
GZMA--F2R IS DONE
LTA_LTB--LTBR IS

In [ ]:
pip freeze